<a href="https://colab.research.google.com/github/aiuser53/test/blob/main/MDX_Colab_(TR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color="Red">Yeni UVR modelleri ile MDX Colab</font><br>

Tüm modelleri otomatik olarak indirilir. <br>Çeşitli modeller için manuel models.py değiştirmeye gerek yoktur.

<sup><br>"vocals.onnx bulunamadı" sorunu yaşıyorsanız ForceUpdate'i kontrol edin veya GDrive'ınızdan MDX_Colab klasörünü silin, oturumu sonlandırın ve baştan başlayın.

<sup><br>Sağlanan modeller [Kuielab](https://github.com/kuielab/mdx-net-submission/), [UVR](https://github.com/Anjok07/ultimatevocalremovergui/) ve [Kim](https://github.com/KimberleyJensen/) tarafından <br> (UVR'yi destekleyebilirsiniz [buradan](https://www.buymeacoffee.com/uvr5/vip-model-download-instructions) ve [buradan](https://boosty.to/uvr)).</sup></br>
<sup><br>Not defteri [Audio Hacker](https://www.youtube.com/channel/UC0NiSV1jLMH-9E09wiDVFYw/), Audio Separation topluluğu tarafından değiştirilmiştir.</sup></br>



In [ ]:
initialised = True
from time import sleep
from google.colab import output
from google.colab import drive

import sys
import os
import shutil
import psutil
import glob

ai = 'https://github.com/kae0-0/Colab-for-MDX_B'
ai_version = 'https://github.com/kae0-0/Colab-for-MDX_B/raw/main/v'
onnx_list = 'https://raw.githubusercontent.com/kae0-0/Colab-for-MDX_B/main/onnx_list'
#@title MDX'i başlatma
#@markdown Drive'a bağlama
drive_bağla = True #@param {type:"boolean"}
#@markdown Bağlama yolu (yalnızca mount_to_drive işaretliyse kullanılır)
Bağlama yolu = '/content/drive/MyDrive' #@param ["snippets:","/content/drive/MyDrive","/content/drive/Shareddrives/<your shared drive name>", "/content/drive/Shareddrives/Shared Drive"]{allow-input: true}
güçlü_güncelleme = False #@param {type:"boolean"}
class h:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
def get_size(bytes, suffix='B'): # read ram
    global svmem
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f'{bytes:.2f}{unit}{suffix}'
        bytes /= factor
    svmem = psutil.virtual_memory()
def console(t):
    get_ipython().system(t)
def LinUzip(file): # unzip call linux, force replace
    console(f'unzip -o {file}')
#-------------------------------------------------------
def getONNX():
    console(f'wget {onnx_list} -O onnx_list')
    _onnx = open("onnx_list", "r")
    _onnx = _onnx.readlines()
    os.remove('onnx_list')
    for model in _onnx:
        _model = sanitize_filename(os.path.basename(model))
        console(f'wget {model}')
        LinUzip(_model)
        os.remove(_model)


def getDemucs(_path):
    #https://dl.fbaipublicfiles.com/demucs/v3.0/demucs_extra-3646af93.th
    root = "https://dl.fbaipublicfiles.com/demucs/v3.0/"
    model = {
        'demucs_extra': '3646af93'
    }
    for models in zip(model.keys(),model.values()):
        console(f'wget {root+models[0]+"-"+models[1]}.th -O {models[0]}.th')
    for _ in glob.glob('*.th'):
        if os.path.isfile(os.path.join(os.getcwd(),_path,_)):
            os.remove(os.path.join(os.getcwd(),_path,_))
        shutil.move(_,_path)
def mount(gdrive=False):
    if gdrive:
        try:
            drive.mount("/content/drive", force_remount=True)
        except:
            drive._mount("/content/drive", force_remount=True)
    else:
        pass
def toPath(path='local'):
    if path == 'local':
        os.chdir('/content')
    elif path == 'gdrive':
        os.chdir(mount_path)

def update():
    with h():
        console(f'wget {ai_version} -O nver')
    f = open('nver', 'r')
    nver = f.read()
    f = open('v', 'r')
    cver = f.read()
    if nver != cver or ForceUpdate:
        print('Yeni güncelleme bulundu! {}'.format(nver))
        os.chdir('../')
        print('yapay zeka güncelleniyor...',end=' ')
        with h():
            console(f'git clone {ai} temp_MDX_Colab')
            console('cp -a temp_MDX_Colab/* MDX_Colab/')
            console('rm -rf temp_MDX_Colab')
        print('bitti')
        os.chdir('MDX_Colab')
        print('Modeller yenileniyor... (4 dakika)', end=' ')
        with h():
            getDemucs('model/')
            getONNX()
        print('bitti')
        output.clear()
        os.remove('v')
        os.rename("nver",'v')
        #os.chdir(f'{os.path.join(mount_path,"MDX_Colab")}')
    else:
        os.remove('nver')
        print('En son sürüm kullanılıyor.')

def past_installation():
    return os.path.exists('MDX_Colab')

def LoadMDX():
    console(f'git clone {ai} MDX_Colab')

#-------------------------------------------------------
# install requirements
print('Bağımlılıkları yüklemek 45 saniye sürecek...',end=' ')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('command not found') >= 0:
    svmem = psutil.virtual_memory()
    gpu_runtime = False
    with h():
        console('pip3 -q install onnxruntime==1.14.1')
else:
    gpu_runtime = True
    with h():
        console('pip3 -q install onnxruntime-gpu==1.14.1')
with h():
    deps = [
            'pathvalidate',
            'youtube-dl',
            'django'
    ]
    for dep in deps:
        console('pip3 -q install {}'.format(dep))
# import modules
#console('pip3 -q install torch==1.13.1')
console('pip3 -q install soundfile==0.12.1')
console('pip3 -q install librosa==0.9.1')
from pathvalidate import sanitize_filename
print('bitti')
if not gpu_runtime:
    print(f'GPU çalışma zamanı devre dışı bırakıldı. Sende olan {get_size(svmem.total)} RAM.\İşlemler inanılmaz derecede yavaş olacaktır. 😈')
elif gpu_info.find('Tesla T4') >= 0:
    print('Tesla T4 GPU nuz var. (hızlar 10-25 it/s civarındadır)')
elif gpu_info.find('Tesla P4') >= 0:
    print('Tesla P4 GPU nuz var. (hızlar 8-22 it/s civarındadır)')
elif gpu_info.find('Tesla K80') >= 0:
    print('Tesla K80 GPU nuz var. (Bu yaygın gpu dur, hızları 2-10 it/s civarındadır)')
elif gpu_info.find('Tesla P100') >= 0:
    print('Tesla P100 GPU nuz var. (Bu, en hızlı ikinci gpu dur, hızlar 15-42 it/s civarındadır)')
else:
    if gpu_runtime:
        print('Bilinmeyen bir GPU nuz var. Lütfen aldığınız GPU yu bildirin!')
        ! nvidia-smi
#console('pip3 -q install demucs')
#-------------------------------------------------------
# Scripting
mount(mount_to_drive)
toPath('gdrive' if mount_to_drive else 'local')
#MDX'in var olup olmadığını kontrol edin
if not past_installation():
    print('İlk kez kurulum yaklaşık 3-6 dakika sürecektir.\nBu, yaklaşık 2-3 GB Boş Gdrive alanı gerektirir.\nLütfen yükleme işlemini kesintiye uğratmamaya çalışın!!!')
    print('yapay zeka yükleniyor...',end=' ')
    with h():
        LoadMDX()
    os.chdir('MDX_Colab')
    print('bitti')

    print('modeller yükleniyor...',end=' ')
    with h():
        getDemucs('model/')
        getONNX()
    if os.path.isfile('onnx_list'):
        os.remove('onnx_list')
    print('bitti')

else:
    os.chdir('MDX_Colab')
    update()

################
#outro
print('Başarılı!')

#<font color="Red">Çıkarım</font>

In [9]:
#@markdown ### Parça listesi yazdırma
for i in glob.glob('tracks/*'):
    print(os.path.basename(i))

vocals.wav
invert_vocals.wav
test.wav
Butterfly.wav


In [29]:
if not 'initialised' in globals():
    raise NameError('Please run the first cell first!! #scrollTo=H_cTbwhVq4K6')

#import all models metadata
import json
with open('model_data.json', 'r') as f:
  model_data = json.load(f)

# Modifiable variables
tracks_path = 'tracks/'
separated_path = 'separated/'

#@markdown ### giriş parça:
#@markdown Herhangi bir bağlantı/dosya adı girin (Şarkılarınızı parçalar klasörüne yükleyin)
track = "Butterfly.wav" #@param {type:"string"}

#@markdown ---
#@markdown ### modeller
ONNX = "MDX-UVR Ins Model Full Band new (HQ_3)" #@param ["off", "Karokee", "Karokee_AGGR", "Karokee 2", "baseline", "MDX-UVR Ins Model 415", "MDX-UVR Ins Model 418", "MDX-UVR Ins Model 464", "MDX-UVR Ins Model 496 - inst main-MDX 2.1", "Kim ft other instrumental model", "MDX-UVR Vocal Model 406", "MDX-UVR Vocal Model 427", "MDX-UVR-Kim Vocal Model (old)", "MDX-UVR-Kim Vocal Model 2 (Kim vocal 2)", "MDX-UVR Ins Model Full Band 292", "MDX-UVR Ins Model Full Band 403", "MDX-UVR Ins Model Full Band 450 (HQ_1)", "MDX-UVR Ins Model Full Band 498 (HQ_2)", "MDX-UVR Ins Model Full Band new (HQ_3)", "(de)Reverb HQ By FoxJoy"]
Demucs = 'off'#@param ["off","demucs_extra"]

#@markdown ---
#@markdown ### Parametreler
denoise = False #@param {type:"boolean"}
normalise = True #@param {type:"boolean"}
#getting values from model_data.json related to ONNX var (model folder name)
amplitude_compensation = model_data[ONNX]["compensate"]
dim_f = model_data[ONNX]["mdx_dim_f_set"]
dim_t = model_data[ONNX]["mdx_dim_t_set"]
n_fft = model_data[ONNX]["mdx_n_fft_scale_set"]

algoritmayı_karıştır = 'min_mag' #@param ["default","min_mag","max_mag"]
chunks = 55 #@param {type:"slider", min:1, max:55, step:1}
shifts = 10 #@param {type:"slider", min:0, max:10, step:0.1}

##validate values
track = track if 'http' in track else tracks_path+track
normalise = '--normalise' if normalise else ''
denoise = '--denoise' if denoise else ''

if ONNX == 'off':
    pass
else:
    ONNX = 'onnx/'+ONNX
if Demucs == 'off':
    pass
else:
    Demucs = 'model/'+Demucs+'.th'
#@markdown ---
#@markdown ### Gövdeler
basslar = False #@param {type:"boolean"}
davullar = False #@param {type:"boolean"}
diğerleri = False #@param {type:"boolean"}
vocaller = True #@param {type:"boolean"}
#@markdown ---
#@markdown ### Sapları karışıma doğru ters çevirin
bassı_ters_çevir = False #@param {type:"boolean"}
davulları_ters_çevir = False #@param {type:"boolean"}
diğerleri_ters_çevir = False #@param {type:"boolean"}
vocalleri_ters_çevir = True #@param {type:"boolean"}
invert_stems = []
stems = []
if basslar:
    stems.append('b')
if davullar:
    stems.append('d')
if diğerleri:
    stems.append('o')
if vocaller:
    stems.append('v')

invert_stems = []
if bassı_ters_çevir:
    invert_stems.append('b')
if davulları_ters_çevir:
    invert_stems.append('d')
if diğerleri_ters_çevir:
    invert_stems.append('o')
if vocalleri_ters_çevir:
    invert_stems.append('v')

margin = 44100

###
# incompatibilities
###

console(f"python main.py --n_fft {n_fft} --dim_f {dim_f} --dim_t {dim_t} --margin {margin} -i \"{track}\" --mixing {algoritmayı_karıştır} --onnx \"{ONNX}\" --model {Demucs} --shifts {round(shifts)} --stems {''.join(stems)} --invert {''.join(invert_stems)} --chunks {chunks} --compensate {amplitude_compensation} {normalise} {denoise}")

Loading onnx model... done
Processing base: 100% 1/1 [00:18<00:00, 18.60s/it]
 >> done

--------------------
Inferences finished!
--------------------
Exporting vocals... done
--------------------
Inverting and exporting vocals... done
--------------------
Successfully completed music demixing.
Total time: 30.3s


*Önerilen modeller*

Enstrümanlar için:
- HQ_2 (en iyi kalite; normal vs min_mag daha az çamurlu, daha fazla vokal kalıntısı)

- kim ft diğer (HQ modelleri ile vokal kalıntılarınız varsa, ancak her zaman değil)

- inst 3/464 (hala biraz gürültülüyse)

- 1.05 v. comp. ile HQ_1 (1.033 v. ile HQ_2'ye kıyasla daha az çamurlu, ancak daha fazla vokal kalıntısı, bunu seviyorum)

Vocaller için:
- kim vocal (vokal gövdede daha az enstrümantal kalıntı)
<br>veya alternatif olarak
- 427
- 406 (yakında)

Baş vokal için:
- Karaokee 2

back vokaller için:
- [HP_KAROKEE-MSB2-3BAND-3090](https://colab.research.google.com/drive/16Q44VBJiIrXOgTINztVDVeb0XKhLKHwl?usp=sharing)

<br>


Değişiklikler
<br><br>
Kim'in vokal modeli sadece bir noktada daha yeni bir modele sahip olduğu için "eski", ancak [SDR] (https://mvsep.com/quality_checker/leaderboard.php?ensemble=0) tarafından 12.03.23 için halka açık olarak değerlendirilen en iyi tek vokal modeliydi (şimdi sadece mvsep1.ru'daki MDX23 tarafından geçildi).

<sup>292 tam bant beta enstrümantal modeli eklendi (14,7 kHz kesme yok). 292 dönemi zaten [aşıldı](https://mvsep.com/quality_checker/leaderboard2.php?&sort=instrum&page=20) Demucs 4 ft modeli. 309 ve 337 epokları halihazırda inst 1 SDR'den daha iyidir.

<sup>Yeni eklenen 403 tam bant beta enstrümantal model (14.7kHz kesme yok) - her durumda olmasa da çoğu durumda 293'ten daha iyi olmalıdır.<br></sup>
Son tam bant 450 (HQ_1) enstrümantal model eklendi (zaman zaman inst3'ten daha fazla vokal kalıntısı bırakabilir, ancak gerçekten denemeye değer).<br>
Yeni 498 dönemi eklendi (HQ_2) "uzun süren vokalleri kaldırmada daha iyi"<br>
Kim diğer ft modeli eklendi (en iyi SDR, hatta 464'e karşı, ancak kesme var), 16. noktayı okuyun.<br>
Eski "Karokee" ve "Karokee_AGGR" modellerini geri döndürdüm, ancak bunlar eski modeller (belki de işe yarayacaklar), montajdan sonra MDX_Colab klasörünüzde zaten varlar.<br>
Crusty Crab tarafından Python 3.10 düzeltmesi.<br>
jarredou tarafından denoiser ve model parametre seçici. <br>
Karaokee 2 modeli. Listeye eklemeyi unutmuşum.<br>
jarredou tarafından Torch 2.0 düzeltmesi. Colab yükleme hızlandırma.<br>
Meşale düzeltildi.

vocals.onnx hatasının devam etmesi durumunda - elle yükleyin [model dosyaları](https://docs.google.com/document/d/17fjNvJzj8ZGSer7c7OFe_CNfUKbAxEh_OBv94ZdRG5c/edit#heading=h.aa2xhwp434) dosyalar.